NLP ANALISIS SENTIMEN

*IMPORT LIBRARY*

In [10]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import nltk

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from wordcloud import WordCloud

nltk.download('stopwords')
nltk.download('punkt')

df = pd.read_csv("twitter_toxic_tweets.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


CASE FOLDING

In [11]:
cf = df['tweet'].str.lower()

CLEANING

In [12]:
clean = df['tweet'].str.replace(r'[^a-z\s]', '', regex=True)

TOKENIZING & STOPWORD REMOVAL

In [13]:
# Stopword Indonesia
stop_words = set(stopwords.words('indonesian'))
def stopword_removal(tokens):
    return [word for word in tokens if word not in stop_words]
df['tweet_tokens'] = df['tweet'].str.split()
df['tweet_tokens'] = df['tweet_tokens'].apply(stopword_removal)

STEMMING

In [14]:
stemmer = SnowballStemmer("english")

def stemming_nltk(tokens):
    return [stemmer.stem(word) for word in tokens]

df['tweet_stemmed'] = df['tweet_tokens'].apply(stemming_nltk)


TF - IDF

In [15]:
# Gabungkan hasil stemming menjadi teks
df['tweet_final'] = df['tweet_stemmed'].apply(lambda x: ' '.join(x))
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['tweet_final'])
y = df['label']


ANALISIS SENTIMEN

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Akurasi Model:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Akurasi Model: 0.9457218833098702
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5937
           1       0.91      0.27      0.41       456

    accuracy                           0.95      6393
   macro avg       0.93      0.63      0.69      6393
weighted avg       0.94      0.95      0.93      6393



UJI TEKS BARU 

In [17]:
# Uji Teks Baru
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Kalimat uji
kalimat_baru = "Permainan timnas sangat memuaskan"

# Preprocessing
kalimat_clean = preprocess_text(kalimat_baru)

# TF-IDF
kalimat_tfidf = tfidf.transform([kalimat_clean])

# Prediksi sentimen
hasil = model.predict(kalimat_tfidf)

# Output
if hasil[0] == 1:
    print("Sentimen: POSITIF")
else:
    print("Sentimen: NEGATIF")


Sentimen: NEGATIF
